# Simple Multi-Agent Debate

참고: https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from dotenv import load_dotenv
load_dotenv()

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

## Tools

In [2]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": str(api_key), "cx": str(search_engine_id), "q": str(query), "num": str(num_results)}

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

## Agents

In [3]:
search_agent = AssistantAgent(
    name="Google_Search_Agent",
    model_client=OpenAIChatCompletionClient(model='gpt-4o-mini'),
    tools=[google_search],
    description="Google Search Agent",
    system_message="""You are a helfpul AI assistant. Solve tasks using your tools. speak in korean.
    Search Google for information, returns top 2 results with a snippet and body content. speak in korean."""
)

man_40_accountant_agent = AssistantAgent(
    name="Man_50_Accountant_Agent",
    model_client=OpenAIChatCompletionClient(model='gpt-4o-mini'),
    description="50 ages amn Accountant Panel",
    system_message="""You are a friendly AI. 
You are a male accountant in your 40s. Represent the general characteristics of your age, gender, and occupation.
State your position on the given topic. 
Please present a one-sided argument rather than a neutral argument.
State your opinion in as much detail and persuasive as possible. Speak in Korean.""",
)

woman_20_college_student_agent = AssistantAgent(
    name="Woman_20_College_Student_Agent",
    model_client=OpenAIChatCompletionClient(model='gpt-4o-mini'),
    description="20 ages Woman College Student Panel",
    system_message="""You are a friendly AI. 
You are a woman college student in your 20s. Represent the general characteristics of your age, gender, and occupation.
State your position on the given topic. 
Please present a one-sided argument rather than a neutral argument.
State your opinion in as much detail and persuasive detail as possible. Speak in Korean."""
)

moderator_agent = AssistantAgent(
    name="Topic_Moderator_Agent",
    model_client=OpenAIChatCompletionClient(model='gpt-4o-mini'),
    description="Dibate Moderator",
    system_message="""You are a friendly AI. 
Summarize the various opinions above and state your conflicting opinions. Speak in Korean.
When you done with generating the opinion, reply with TERMINATE."""
)

# Agent 중 누군가가 'TERMINATE' 라고 발언하면 토론 종료  
termination = TextMentionTermination("TERMINATE")
# RoundRobinGroupChat 은 토론 참가자가 돌아가면서 발언하는 방식 
team = RoundRobinGroupChat([
    search_agent, 
    man_40_accountant_agent, 
    woman_20_college_student_agent, 
    moderator_agent], 
    termination_condition=termination
)

In [4]:
result = await team.run(task="서부지법 폭동 사태에 대한 기사를 참고하여 의견을 말하세요.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='서부지법 폭동 사태에 대한 기사를 참고하여 의견을 말하세요.', type='TextMessage'), ToolCallRequestEvent(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=22), content=[FunctionCall(id='call_W81Gx7Msh2X2AoD6vZ3QX5He', arguments='{"query":"서부지법 폭동 사태"}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Google_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': \'“서부지법 사태, 2021년 미국 의회 폭동 연상시켜”\', \'link\': \'https://www.chosun.com/national/incident/2025/01/19/CBBUGNSR6JHTZJVEJEFV5IEYRI/\', \'snippet\': \'1 day ago ... 미국에서도 2021년 1월 6일(현지 시각) 이번 서부지법 사태와 유사한 미 국회의사당 점거 폭동이 있었다. 이날은 2020년 미 대선에서 승리한 조 바이든\\xa0...\', \'body\': \'“서부지법 사태, 2021년 미국 의회 폭동 연상시켜”\'}, {\'title\': \'경찰 "서부지법 폭동 전원 구속수사‥법치주의에 대한 도전"\', \'link\': \'https://imnews.imbc.com/news/2025/society/article/6678552_36718.html\', \'snippet\': \'1 day ago ... 윤석열 

In [5]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="서부지법 폭동 사태에 대한 기사를 참고하여 의견을 말하세요."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='서부지법 폭동 사태에 대한 기사를 참고하여 의견을 말하세요.' type='TextMessage'
source='Google_Search_Agent' models_usage=RequestUsage(prompt_tokens=142, completion_tokens=27) content=[FunctionCall(id='call_yiFeQ6vIPrpcPCYMkS8hjgSk', arguments='{"query":"서부지법 폭동 사태","num_results":2}', name='google_search')] type='ToolCallRequestEvent'
source='Google_Search_Agent' models_usage=None content=[FunctionExecutionResult(content='[{\'title\': \'“서부지법 사태, 2021년 미국 의회 폭동 연상시켜”\', \'link\': \'https://www.chosun.com/national/incident/2025/01/19/CBBUGNSR6JHTZJVEJEFV5IEYRI/\', \'snippet\': \'1 day ago ... 미국에서도 2021년 1월 6일(현지 시각) 이번 서부지법 사태와 유사한 미 국회의사당 점거 폭동이 있었다. 이날은 2020년 미 대선에서 승리한 조 바이든\\xa0...\', \'body\': \'“서부지법 사태, 2021년 미국 의회 폭동 연상시켜”\'}, {\'title\': \'경찰 "서부지법 폭동 전원 구속수사‥법치주의에 대한 도전"\', \'link\': \'https://imnews.imbc.com/news/2025/society/article/6678552_36718.html\', \'snippet\': \'1 day ago ... 윤석열 대통령 구속에 반대하는 극렬 지지자들이 서울서부지방법원에 난입해 폭력 사태를 일으킨 데 대해 경찰이 "주동자는 물론 불법 행위자 전

In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="차별금지법안은 필요한 법인가?"))  # Stream the messages to the console.

---------- user ----------
차별금지법안은 필요한 법인가?
---------- Google_Search_Agent ----------
[FunctionCall(id='call_uXlf7GnDsB2PK5JIbfFjD9aH', arguments='{"query":"차별금지법안 필요성","num_results":2}', name='google_search')]
---------- Google_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': \'차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 - BBC ...\', \'link\': \'https://www.bbc.com/korean/news-66065811\', \'snippet\': \'Aug 2, 2023 ... 차별금지법 제정의 필요성을 역설하는 이들은 개별적 차별금지법으로는 ... 오래전부터 포괄적 차별금지법안이 발의돼 왔지만, 구체적인 논의나\\xa0...\', \'body\': "차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 - BBC News 코리아 BBC News, 코리아 내용 보기 섹션 뉴스 비디오 라디오 다운로드 TOP 뉴스 뉴스 비디오 라디오 다운로드 TOP 뉴스 차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 사진 출처, 차별금지법제정연대 홈페이지 사진 설명, 차별금지법제정연대가 1일 충남 내포혁신플랫폼에서 \'성평등 책 금서 요구\'를 비판하는 토론회를 열었다 기사 관련 정보 기자, 리차드 김 기자, BBC 코리아 2023년 8월 2일 충청남도 공공도서관에 비치된 성교육·성평등 관련 도서를 두고 갑론을박이 이어지면서 이른바 \'차별금지법\'을 둘러싼 갈등도 재점화되는 분위기다. 이번 사태는 최근 충남도의회에서 \'성교육 도서\'에 대한 공식적인 문제 제기 발언이 나오면서 확산했다. 지민규 도의원(국민의힘)은 지난달 25일 열린 본회의 긴급현안질문에

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='차별금지법안은 필요한 법인가?', type='TextMessage'), ToolCallRequestEvent(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=27), content=[FunctionCall(id='call_uXlf7GnDsB2PK5JIbfFjD9aH', arguments='{"query":"차별금지법안 필요성","num_results":2}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Google_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': \'차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 - BBC ...\', \'link\': \'https://www.bbc.com/korean/news-66065811\', \'snippet\': \'Aug 2, 2023 ... 차별금지법 제정의 필요성을 역설하는 이들은 개별적 차별금지법으로는 ... 오래전부터 포괄적 차별금지법안이 발의돼 왔지만, 구체적인 논의나\\xa0...\', \'body\': "차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 - BBC News 코리아 BBC News, 코리아 내용 보기 섹션 뉴스 비디오 라디오 다운로드 TOP 뉴스 뉴스 비디오 라디오 다운로드 TOP 뉴스 차별금지법: 17년째 국회서 쳇바퀴...제정되지 않는 진짜 이유 사진 출처, 차별금지법제정연대 홈페이지 사진 설명, 차별금지법제정연대가 1일 충남 내포혁신플랫폼에서 \'성평등 책 금서 요구\'를 비판하는 토론회